In [ ]:
import pandas as pd
import numpy as np
import math
import sys

# Creating Dataset

In [ ]:
df = pd.read_csv("data_part_clean4.csv")
df_full = df[["INCIDENT_DATETIME", "ZIPCODE", "NEIGHBORHOOD"]]
df_full["BOROUGH"] = "Manhattan"

In [ ]:
# VARIABLES
neighborhood_or_zipcode = ["BOROUGH", "NEIGHBORHOOD", "ZIPCODE"]
delta_aggregs = [1,4,12] # number of hours for aggregations
delta_weeks = [1,2,3,4] # number of weeks for lags
delta_years = [0,1,2] # number of years for weekly lags
forecasting_horizon = 1 # forecasting horizon
roll_split = 4 # divider for train and test set
hours_year= 8736

In [ ]:
shift_delta_list=[1] # list of all the shifts
for delta_aggreg in delta_aggregs:
    for delta_year in delta_years:
        for delta_week in delta_weeks:
            shift_delta_list.append(int((delta_year*hours_year+delta_week*7*24)/delta_aggreg))

In [ ]:
# creating variables for shifts
count_lag_columns = []
neig_lag_columns = []
bor_lag_columns = []
zip_lag_columns = []
for element in shift_delta_list:
    count_lag_columns.append("COUNT_LAG_"+str(element))
    neig_lag_columns.append("NEIG_LAG_"+str(element))
    bor_lag_columns.append("BOR_LAG_"+str(element))
    zip_lag_columns.append("ZIP_LAG_"+str(element))

In [ ]:
# creating population density reference for additional variables
# hourly population density for manhattan districts is taken from Manhattan Population Explorer (https://github.com/citrusvanilla/manhattanpopulationexplorer)
density_reference = pd.read_csv("density_reference3.csv", delimiter=";") # load reference table
columns_list = density_reference.columns.tolist()[1:] # create list of column names
density_reference = pd.melt(density_reference, id_vars="nid", value_vars=columns_list, var_name="HOUR", value_name="POPULATION") # wide to long format
density_reference.HOUR = pd.to_numeric(density_reference.HOUR) # reformat datatype
density_reference.POPULATION = pd.to_numeric(density_reference.POPULATION) # reformat datatype
density_reference["WEEKDAY"] = np.floor(density_reference.HOUR/24) # new column for weekday number
density_reference["HOUR"] = density_reference.HOUR%24 # make hours count to 23 instead of 167
density_reference["1H"] = (density_reference.HOUR//1)*1 # add aggregation sum
density_reference["4H"] = (density_reference.HOUR//4)*4 # add aggregation sum
density_reference["12H"] = (density_reference.HOUR//12)*12 # add aggregation sum
density_reference_1H = density_reference.groupby(["nid", "WEEKDAY", "1H"]).POPULATION.sum().reset_index() # create reference table
density_reference_4H = density_reference.groupby(["nid", "WEEKDAY", "4H"]).POPULATION.sum().reset_index() # create reference table
density_reference_12H = density_reference.groupby(["nid", "WEEKDAY", "12H"]).POPULATION.sum().reset_index() # create reference table

In [ ]:
# creating tables for NEIGHBORHOOD and ZIPCODE and saving them in df_list
df_list = {}
for neig_zip in neighborhood_or_zipcode:
    for aggregation_size in delta_aggregs:
        print("working on creating",neig_zip, str(aggregation_size)+"H")
        df = df_full[["INCIDENT_DATETIME", neig_zip]]
        df["COUNT"] = pd.Series([1 for x in range(len(df.index))])

        df['INCIDENT_DATETIME'] = pd.to_datetime(df.INCIDENT_DATETIME)

        df['INCIDENT_DATETIME'] = df.INCIDENT_DATETIME.dt.floor(str(aggregation_size)+"H")
        df = df.groupby([neig_zip, 'INCIDENT_DATETIME']).COUNT.count().reset_index()
        time = pd.date_range(df.INCIDENT_DATETIME.min(), df.INCIDENT_DATETIME.max(), freq=str(aggregation_size)+"H")

        full = []
        for neigh in df[neig_zip].unique():
            full.append(pd.DataFrame({
                'INCIDENT_DATETIME': time, neig_zip: [neigh]*len(time)
        }))
        full = pd.concat(full).reset_index(drop=True)

        full = full.merge(df, on=[neig_zip, 'INCIDENT_DATETIME'], how='left')
        full = full.fillna(0)
        full.COUNT = full.COUNT.astype("int32")
        
        name = "df_"+str(neig_zip)+"_"+str(aggregation_size)+"H"
        df_list[name] = full
print("done")

In [ ]:
# add sin and cos for cyclical feature encoding
for element in list(df_list.keys()):
    print("working on sin/cos for",element)
    df_list[element]["HOUR"] = df_list[element]["INCIDENT_DATETIME"].dt.hour
    df_list[element]["WEEKDAY"] = df_list[element]["INCIDENT_DATETIME"].dt.weekday
    df_list[element]["MONTH"] = df_list[element]["INCIDENT_DATETIME"].dt.month

    df_list[element]["HOUR_norm"] = 2 * math.pi * df_list[element]["HOUR"] / df_list[element]["HOUR"].max()
    df_list[element]["HOUR_cos"] = np.cos(df_list[element]["HOUR_norm"])
    df_list[element]["HOUR_sin"] = np.sin(df_list[element]["HOUR_norm"])

    df_list[element]["WEEKDAY_norm"] = 2 * math.pi * df_list[element]["WEEKDAY"] / df_list[element]["WEEKDAY"].max()
    df_list[element]["WEEKDAY_cos"] = np.cos(df_list[element]["WEEKDAY_norm"])
    df_list[element]["WEEKDAY_sin"] = np.sin(df_list[element]["WEEKDAY_norm"])

    df_list[element]["MONTH_norm"] = 2 * math.pi * df_list[element]["MONTH"] / df_list[element]["MONTH"].max()
    df_list[element]["MONTH_cos"] = np.cos(df_list[element]["MONTH_norm"])
    df_list[element]["MONTH_sin"] = np.sin(df_list[element]["MONTH_norm"])

# adding poopulation density
    if "NEIGHBORHOOD_1H" in element:
        df_list[element].NEIGHBORHOOD = df_list[element].NEIGHBORHOOD.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_1H, how="left", left_on=["NEIGHBORHOOD", "HOUR", "WEEKDAY"], right_on = ["nid", "1H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "1H"])
    elif "NEIGHBORHOOD_4H" in element:
        df_list[element].NEIGHBORHOOD = df_list[element].NEIGHBORHOOD.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_4H, how="left", left_on=["NEIGHBORHOOD", "HOUR", "WEEKDAY"], right_on = ["nid", "4H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "4H"])    
    elif "NEIGHBORHOOD_12H" in element:
        df_list[element].NEIGHBORHOOD = df_list[element].NEIGHBORHOOD.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_12H, how="left", left_on=["NEIGHBORHOOD", "HOUR", "WEEKDAY"], right_on = ["nid", "12H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "12H"])
    elif "ZIPCODE_1H" in element:
        df_list[element].ZIPCODE = df_list[element].ZIPCODE.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_1H, how="left", left_on=["ZIPCODE", "HOUR", "WEEKDAY"], right_on = ["nid", "1H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "1H"])
    elif "ZIPCODE_4H" in element:
        df_list[element].ZIPCODE = df_list[element].ZIPCODE.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_4H, how="left", left_on=["ZIPCODE", "HOUR", "WEEKDAY"], right_on = ["nid", "4H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "4H"])    
    elif "ZIPCODE_12H" in element:
        df_list[element].ZIPCODE = df_list[element].ZIPCODE.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_12H, how="left", left_on=["ZIPCODE", "HOUR", "WEEKDAY"], right_on = ["nid", "12H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "12H"])
    elif "BOROUGH_1H" in element:
        df_list[element].BOROUGH = df_list[element].BOROUGH.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_1H, how="left", left_on=["BOROUGH", "HOUR", "WEEKDAY"], right_on = ["nid", "1H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "1H"])
    elif "BOROUGH_4H" in element:
        df_list[element].BOROUGH = df_list[element].BOROUGH.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_4H, how="left", left_on=["BOROUGH", "HOUR", "WEEKDAY"], right_on = ["nid", "4H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "4H"])    
    elif "BOROUGH_12H" in element:
        df_list[element].BOROUGH = df_list[element].BOROUGH.astype(str)
        df_list[element] = pd.merge(df_list[element], density_reference_12H, how="left", left_on=["BOROUGH", "HOUR", "WEEKDAY"], right_on = ["nid", "12H", "WEEKDAY"])
        df_list[element] = df_list[element].drop(columns=["nid", "12H"])

print("done")

In [ ]:



#adding lag
for element in list(df_list.keys()):
    if "NEIGHBORHOOD" in element:
        print("working on lag for",element)
        # add lag
        for count_lag, neig_lag, shift_delta in zip(count_lag_columns, neig_lag_columns, shift_delta_list):
            df_list[element][neig_lag] = df_list[element]["NEIGHBORHOOD"].shift(shift_delta)
            df_list[element][count_lag] = df_list[element]["COUNT"].shift(shift_delta)

        # Lag Bleeding Correction
        for count_lag, neig_lag in zip(count_lag_columns, neig_lag_columns):
            df_list[element][count_lag] = np.where(df_list[element]["NEIGHBORHOOD"] != df_list[element][neig_lag], np.nan, df_list[element][count_lag])
            df_list[element][neig_lag] = np.where(df_list[element]["NEIGHBORHOOD"] != df_list[element][neig_lag], np.nan, df_list[element][neig_lag])

    elif "ZIPCODE" in element:
        print("working on lag for",element)
        # add lag
        for count_lag, zip_lag, shift_delta in zip(count_lag_columns, zip_lag_columns, shift_delta_list):
            df_list[element][zip_lag] = df_list[element]["ZIPCODE"].shift(shift_delta)
            df_list[element][count_lag] = df_list[element]["COUNT"].shift(shift_delta)

        # Lag Bleeding Correction
        for count_lag, zip_lag in zip(count_lag_columns, zip_lag_columns):
            df_list[element][count_lag] = np.where(df_list[element]["ZIPCODE"] != df_list[element][zip_lag], np.nan, df_list[element][count_lag])
            df_list[element][zip_lag] = np.where(df_list[element]["ZIPCODE"] != df_list[element][zip_lag], np.nan, df_list[element][zip_lag])

    elif "BOROUGH" in element:
        print("working on lag for",element)
        # add lag
        for count_lag, bor_lag, shift_delta in zip(count_lag_columns, bor_lag_columns, shift_delta_list):
            df_list[element][bor_lag] = df_list[element]["BOROUGH"].shift(shift_delta)
            df_list[element][count_lag] = df_list[element]["COUNT"].shift(shift_delta)

        # Lag Bleeding Correction
        for count_lag, bor_lag in zip(count_lag_columns, bor_lag_columns):
            df_list[element][count_lag] = np.where(df_list[element]["BOROUGH"] != df_list[element][bor_lag], np.nan, df_list[element][count_lag])
            df_list[element][bor_lag] = np.where(df_list[element]["BOROUGH"] != df_list[element][bor_lag], np.nan, df_list[element][bor_lag])

print("done")

# adding lag mean (MEDIC)
lag_list_1=[] # list of all the shifts
delta_aggregs = [1]
for delta_aggreg in delta_aggregs:
    for delta_year in delta_years:
        for delta_week in delta_weeks:
            lag_list_1.append(int((delta_year*hours_year+delta_week*7*24)/delta_aggreg))
lag_columns_1 = []
for item in lag_list_1:
    lag_columns_1.append("COUNT_LAG_"+str(item))

lag_list_4=[] # list of all the shifts
delta_aggregs = [4]
for delta_aggreg in delta_aggregs:
    for delta_year in delta_years:
        for delta_week in delta_weeks:
            lag_list_4.append(int((delta_year*hours_year+delta_week*7*24)/delta_aggreg))
lag_columns_4 = []
for item in lag_list_4:
    lag_columns_4.append("COUNT_LAG_"+str(item))

lag_list_12=[] # list of all the shifts
delta_aggregs = [12]
for delta_aggreg in delta_aggregs:
    for delta_year in delta_years:
        for delta_week in delta_weeks:
            lag_list_12.append(int((delta_year*hours_year+delta_week*7*24)/delta_aggreg))
lag_columns_12 = []
for item in lag_list_12:
    lag_columns_12.append("COUNT_LAG_"+str(item))


for element in list(df_list.keys()):
    print("working on lag mean for",element)
    df_list[element]["MEAN_LAG_1"] = df_list[element][lag_columns_1].mean(axis=1)
    df_list[element]["MEAN_LAG_4"] = df_list[element][lag_columns_4].mean(axis=1)
    df_list[element]["MEAN_LAG_12"] = df_list[element][lag_columns_12].mean(axis=1)
print("done")


# Forecasting

In [ ]:
import time
from darts import TimeSeries
from darts.models import ExponentialSmoothing, NBEATSModel, AutoARIMA, Theta, ARIMA, NaiveSeasonal, Prophet, LightGBMModel, RandomForest, RNNModel, StatsForecastAutoARIMA, LinearRegressionModel
from sklearn import linear_model
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
import random

In [ ]:
results=[]
results_counter=1

In [ ]:
list_of_dfs = list(df_list.keys())

iterator_model = ["NaiveSeasonal(K=24)", "NaiveSeasonal(K=6)", "NaiveSeasonal(K=2)", 
                    "ExponentialSmoothing()", 
                    "StatsForecastAutoARIMA()", 
                    "linear_model.LinearRegression()",
                    "linear_model.LinearRegression()",
                    "MEDIC",
                    "MLPClassifier()",
                    "MLPClassifier()"]
iterator_model_repeater = ["none", "none", "none", # naive
                            "none", # exponential smoothing
                            "none", # autoARIMA
                            "no_population", # linearregression
                            "with_population", # linearregression
                            "none", #MEDIC
                            "no_population", # MLPClassifier
                            "with_population"] # MLPClassifier
x_columns = [ "HOUR_cos", "HOUR_sin", "WEEKDAY_cos", "WEEKDAY_sin", "MONTH_cos", "MONTH_sin"]
y_columns = ["COUNT"]
date_column = ["INCIDENT_DATETIME"]
medic_columns = ["MEAN_LAG_1", "MEAN_LAG_4", "MEAN_LAG_12"]
population_columns = ["POPULATION"]

for df_element in list_of_dfs: # selecting df from list of dfs
    grouper = "empty" # variable for either NEIGHBORHOOD or ZIPCODE
    if "NEIGHBORHOOD" in df_element:
        grouper = "NEIGHBORHOOD"
    elif "ZIPCODE" in df_element:
        grouper = "ZIPCODE"
    elif "BOROUGH" in df_element:
        grouper = "BOROUGH"

    list_of_groups = list(df_list[df_element][grouper].unique())

    for group_element in list_of_groups:
        series = TimeSeries.from_dataframe(df_list[df_element][df_list[df_element][grouper] == group_element], "INCIDENT_DATETIME", "COUNT")
        series2 = df_list[df_element][df_list[df_element][grouper] == group_element][x_columns+y_columns+date_column]
        series3 = df_list[df_element][df_list[df_element][grouper] == group_element][date_column+y_columns+medic_columns]
        series4 = df_list[df_element][df_list[df_element][grouper] == group_element][x_columns+y_columns+date_column+population_columns]

        series_total_len = len(series)
        series_test_len = round(series_total_len/roll_split)
        series_train_len = series_total_len-series_test_len

        for model_name, model_repeater in zip(iterator_model, iterator_model_repeater): # selecting model from list of models
            
            # optimizer for calculation duration:
            if "1H" in df_element:
                series_test_len_temporary = range(4206, 4206+24+24+24)
            elif "4H" in df_element:
                series_test_len_temporary = range(1052, 1052+6+6+6)
            elif "12H" in df_element:
                series_test_len_temporary = range(350, 350+2+2+2)

            for rolling_iterator in series_test_len_temporary: #############################################
                # status output to console  
                sys.stdout.write("\r")
                sys.stdout.write(str(df_element)+ " group_element: "+str(group_element)+"  iterator: "+str(rolling_iterator+1)+"  model: "+str(model_name))  
                
                if "NaiveSeasonal" in model_name or "ExponentialSmoothing" in model_name or "StatsForecastAutoARIMA" in model_name:
                    start_time = time.time()

                    train, val = series[rolling_iterator:series_train_len+rolling_iterator], series[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon]
                    model = eval(model_name)
                    model.fit(train)
                    prediction = model.predict(len(val))

                    y_real = val.pd_dataframe().iat[0,0]
                    y_predict = prediction.pd_dataframe().iat[0,0]
                    y_Datetime = prediction.pd_dataframe().index[0].to_datetime64()

                    end_time = time.time()
                    duration = end_time-start_time
                
                elif "LinearRegression" in model_name and "no_population" in model_repeater:
                    start_time = time.time()  

                    train, val = series2[rolling_iterator:series_train_len+rolling_iterator], series2[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon]

                    lm = eval(model_name)
                    x = train[x_columns]
                    y = train[y_columns]
                    model = lm.fit(x,y)
                    
                    y_predict = lm.predict(val[x_columns]).tolist()[0][0] 
                    y_real = val[y_columns].iat[0,0]
                    y_Datetime = val[date_column].iat[0,0] 
                    
                    end_time = time.time()
                    duration = end_time-start_time

                elif "LinearRegression" in model_name and "with_population" in model_repeater:
                    start_time = time.time()  

                    train, val = series4[rolling_iterator:series_train_len+rolling_iterator], series4[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon]

                    lm = eval(model_name)
                    x = train[x_columns+population_columns]
                    y = train[y_columns]
                    model = lm.fit(x,y)
                    
                    y_predict = lm.predict(val[x_columns+population_columns]).tolist()[0][0] 
                    y_real = val[y_columns].iat[0,0]
                    y_Datetime = val[date_column].iat[0,0] 
                    
                    end_time = time.time()
                    duration = end_time-start_time
                
                elif "MEDIC" in model_name:
                    start_time = time.time()
                    y_Datetime = "1970-01-01 00:00:00"
                    y_real = 999
                    y_predict = 999
                    if "1H" in df_element:
                        y_Datetime = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][date_column].iat[0,0]
                        y_real = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][y_columns].iat[0,0]
                        y_predict = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][medic_columns].iat[0,0]
                    elif "4H" in df_element:
                        y_Datetime = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][date_column].iat[0,0]
                        y_real = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][y_columns].iat[0,0]
                        y_predict = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][medic_columns].iat[0,1]
                    elif "12H" in df_element:
                        y_Datetime = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][date_column].iat[0,0]
                        y_real = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][y_columns].iat[0,0]
                        y_predict = series3[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon][medic_columns].iat[0,2]

                    end_time = time.time()
                    duration = end_time-start_time
                
                elif "MLPClassifier" in model_name and "no_population" in model_repeater:
                    start_time = time.time()

                    train, val = series2[rolling_iterator:series_train_len+rolling_iterator], series2[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon]

                    np.random.seed(2)
                    model = MLPClassifier()
                    model.fit(train[x_columns], np.ravel(train[y_columns]))
                    
                    y_predict = model.predict(val[x_columns])[0]
                    y_real = val[y_columns].iat[0,0]
                    y_Datetime = val[date_column].iat[0,0] 

                    end_time = time.time()
                    duration = end_time - start_time

                elif "MLPClassifier" in model_name and "with_population" in model_repeater:
                    start_time = time.time()

                    train, val = series4[rolling_iterator:series_train_len+rolling_iterator], series4[series_train_len+rolling_iterator:series_train_len+rolling_iterator+forecasting_horizon]

                    np.random.seed(2)
                    model = MLPClassifier()
                    model.fit(train[x_columns+population_columns], np.ravel(train[y_columns]))
                    
                    y_predict = model.predict(val[x_columns+population_columns])[0]
                    y_real = val[y_columns].iat[0,0]
                    y_Datetime = val[date_column].iat[0,0] 

                    end_time = time.time()
                    duration = end_time - start_time
                
                results.append({"NH/ZIP":df_element, "Grouper":grouper, "Group_Element":group_element, "y_real":y_real, "y_predict":y_predict, "Datetime":y_Datetime, "Model":model_name, "Duration":duration, "Repeater":model_repeater})
            
            print(str(" done"))

        pd.DataFrame(results).to_csv("results_BOR_"+str(results_counter)+"_"+str(df_element)+"_"+str(group_element)+".csv")
        results = []
        results_counter = results_counter +1



